In [8]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

In [9]:
df = pd.read_csv('D:\Productivity\Studying\PMLDL_A1\data\interim\preprocessed.tsv', sep='\t')
df.head()

,Unnamed: 0.1,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,1,1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
2,3,3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
3,4,4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
4,5,5,I'm not gonna have a child... ...with the same...,I'm not going to breed kids with a genetic dis...,0.703185,0.206522,0.950956,0.035846


In [10]:
df.sample(frac=0.8, random_state=13)

,Unnamed: 0.1,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
123423,176381,176381,"if he sees me, and I'm gonna look like that no...","If they get one look at me looking like this, ...",0.630866,0.058824,0.980718,0.000230
320784,458500,458500,"you fuck me because once you have a problem, y...",I'm pissed off because every time you have a p...,0.723548,0.142857,0.999320,0.018898
84650,120912,120912,"Sergeant, why are you breaking my balls here?","sergeant, why do you keep bumping into me?",0.661492,0.065217,0.997737,0.000903
106439,152110,152110,that's the stupidest idea I've ever heard.,That is the worst idea I've ever heard.,0.858453,0.069767,0.999686,0.015469
389618,556394,556394,"trust me, pussycat.","Trust me, kitty Kat.",0.877873,0.047619,0.993290,0.000194
...,...,...,...,...,...,...,...,...
297474,425367,425367,you want to show me what the damn basics are?,"You want to show me, uh, what the heck a footi...",0.785450,0.132075,0.996686,0.008486
209753,299428,299428,"Bury them now, missy?",bury them now?,0.836797,0.318182,0.949059,0.001496
104067,148697,148697,Open the door and throw that bitch out.,open the door and throw it out.,0.835778,0.200000,0.999542,0.021942
39667,56732,56732,I don't know how you're going with your father...,I don't know about you and your father but let...,0.790343,0.043796,0.999340,0.025873


In [11]:
df = df.rename(columns={"reference": "input_text", "translation": "target_text"})
train, test = train_test_split(df, test_size=0.2)
train = pd.DataFrame(train, columns = ['input_text','target_text'])
test = pd.DataFrame(test, columns = ['input_text','target_text'])
train.head()

,input_text,target_text
159775,he doesn't like geeks.,He does not like Geisha.
211169,"and try pointing and shooting at them, not int...","And try to aim and hit them, not me."
377322,and I want you to have one deep inside of you.,But I want you to have one in depth.
12014,Betting's for suckers.,betting is for magicians.
107946,I killed him a week ago.,Just last week I had a slip.


In [12]:
test.head()

,input_text,target_text
97444,"Semantics. You say tomato-- No, motherfucker, ...","you say tomato No, I didn't say ""tomato."""
235611,"We forgot the word processor, but we left you ...","we've forgotten the text processor, but we lef..."
134721,You idiot! You gonna spill hot coffee all over...,"he's covered me with a hot coffee, huh?"
183956,"No, you do... you do that and then that's what...","no, you do this. That's what men do when they ..."
28775,he can have a baby with you if you don't even ...,"How can she have a child by you, when you won'..."


In [13]:
model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False

model_args.do_sample = True
model_args.num_beams = 1
model_args.num_return_sequences = 1
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95
model_args.output_dir = "./models/tany"


In [14]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="eugenesiow/bart-paraphrase",
    args=model_args,
    use_cuda=False,
    use_mps_device = True
)
model.train_model(train, eval_data=test)

KeyboardInterrupt: 

In [ ]:
preds = model.predict(test["input_text"][:5].tolist())

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/strangetany/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/Users/strangetany/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [ ]:
preds

['I want to believe Okwe never fucked you.',
 "Why didn't the mountain provide a pass for sledging.",
 "Loyalty to anybody: I'm Glad you're back in town, Mr. Clinton. Yeah, me too.",
 "He's ashamed of you because you have a blob in your life.",
 'If I had a rape whistle, would you call me a feminist, what should I do?']

In [ ]:
test["input_text"][:5]

93               I can't believe Okwe never fucked you.
57                  I don't need a mountain for chumps.
91    Damn, I'm glad you're back in town. Yeah, me too.
39    he's ashamed of you because you have a blob in...
88            We're gonna send you a rape whistle. Huh?
Name: input_text, dtype: object